In [0]:
# data analysis packages
import pandas as pd
import numpy as np

# loading bar
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

     - 706kB 1.6MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-cxgwz8su/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Dataset")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Loading Datasets

In [0]:
full_benign_path = './full_benign_df.csv'

full_benign_df = pd.read_csv(full_benign_path)
full_benign_df.shape

(555932, 116)

In [0]:
full_mirai_path = './full_mirai_df.csv'

full_mirai_df = pd.read_csv(full_mirai_path)
full_mirai_df.shape

(3668402, 116)

In [0]:
full_bashlite_path = './full_bashlite_df.csv'

full_bashlite_df = pd.read_csv(full_bashlite_path)
full_bashlite_df.shape

(2838272, 116)

In [0]:
#Sampling 1% of each of the three classes

sampled_benign_df = full_benign_df.sample(frac =.01)

sampled_mirai_df = full_mirai_df.sample(frac =.01)

sampled_bashlite_df = full_bashlite_df.sample(frac =.01)

## Fishers Score

In [0]:
dataframes = [full_benign_df, full_mirai_df, full_bashlite_df]

sampled_dfs = [sampled_benign_df, sampled_mirai_df, sampled_bashlite_df]

#have to create a list with feature name and fisher score from it

total_points = (len(full_benign_df.index) + len(full_mirai_df.index) + len(full_bashlite_df.index))

feature_fisher_scores = pd.DataFrame(columns=['feature', 'fisher_score'])

features = []
fs = []
k = 0

for j in tqdm(full_benign_df.columns):
  numerator = 0
  denom = 0
  #for i in dataframes:
  for i in sampled_dfs:
    #pi = len(i.index)  / total_points
    pi = len(i.index)
    #i_frames = [full_benign_df.iloc[:,k], full_mirai_df.iloc[:,k], full_bashlite_df.iloc[:,k]]
    i_frames = [sampled_benign_df.iloc[:,k], sampled_mirai_df.iloc[:,k], sampled_bashlite_df.iloc[:,k]]
    
    full_df = pd.concat(i_frames)

    total_mean = np.mean(full_df)
    i_mean = np.mean((i.iloc[:,k]))

    mean_terms = ((total_mean - i_mean)**2)
    #mean_terms = (abs(total_mean - i_mean))
    
    numerator = (pi*mean_terms) + numerator
    
    sd = np.std((i.iloc[:,k]))
    denom = (pi*((sd)**2)) + denom
    #denom = (pi*(abs(sd))) + denom
  
  k = k + 1
  fisher_score = numerator/denom
  features.append(j)
  fs.append(fisher_score)

feature_fisher_scores['feature'] = features
feature_fisher_scores['fisher_score'] = fs

feature_fisher_scores.head()

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


,feature,fisher_score
0,Unnamed: 0,0.069945
1,MI_dir_L5_weight,0.492729
2,MI_dir_L5_mean,0.591789
3,MI_dir_L5_variance,0.506156
4,MI_dir_L3_weight,0.475006


In [0]:
feature_fisher_scores.sort_values(['fisher_score'], ascending=False).head(10)

,feature,fisher_score
13,MI_dir_L0.01_weight,1.084794
28,H_L0.01_weight,1.084794
14,MI_dir_L0.01_mean,0.766793
29,H_L0.01_mean,0.766779
11,MI_dir_L0.1_mean,0.757560
26,H_L0.1_mean,0.757559
8,MI_dir_L1_mean,0.726792
23,H_L1_mean,0.726792
30,H_L0.01_variance,0.701236
15,MI_dir_L0.01_variance,0.701235


# Models From Reduced Feature Sets

## Preprocessing the data

In [0]:
s_ben_df = sampled_benign_df

s_ben_df['label'] = 0

s_mir_df = sampled_mirai_df

s_mir_df['label'] = 1

s_bash_df = sampled_bashlite_df

s_bash_df['label'] = 2

s_bash_df = s_bash_df.drop(columns=['Unnamed: 0'])
s_mir_df = s_mir_df.drop(columns=['Unnamed: 0'])
s_ben_df = s_ben_df.drop(columns=['Unnamed: 0'])

frames = [s_bash_df, s_mir_df, s_ben_df]
s_combined_df = pd.concat(frames)

s_rand_combined_df = s_combined_df.sample(frac=1).reset_index(drop=True)

s_rand_combined_df.head(10)

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,MI_dir_L0.1_mean,MI_dir_L0.1_variance,MI_dir_L0.01_weight,MI_dir_L0.01_mean,MI_dir_L0.01_variance,H_L5_weight,H_L5_mean,H_L5_variance,H_L3_weight,H_L3_mean,H_L3_variance,H_L1_weight,H_L1_mean,H_L1_variance,H_L0.1_weight,H_L0.1_mean,H_L0.1_variance,H_L0.01_weight,H_L0.01_mean,H_L0.01_variance,HH_L5_weight,HH_L5_mean,HH_L5_std,HH_L5_magnitude,HH_L5_radius,HH_L5_covariance,HH_L5_pcc,HH_L3_weight,HH_L3_mean,HH_L3_std,...,HH_jit_L0.1_variance,HH_jit_L0.01_weight,HH_jit_L0.01_mean,HH_jit_L0.01_variance,HpHp_L5_weight,HpHp_L5_mean,HpHp_L5_std,HpHp_L5_magnitude,HpHp_L5_radius,HpHp_L5_covariance,HpHp_L5_pcc,HpHp_L3_weight,HpHp_L3_mean,HpHp_L3_std,HpHp_L3_magnitude,HpHp_L3_radius,HpHp_L3_covariance,HpHp_L3_pcc,HpHp_L1_weight,HpHp_L1_mean,HpHp_L1_std,HpHp_L1_magnitude,HpHp_L1_radius,HpHp_L1_covariance,HpHp_L1_pcc,HpHp_L0.1_weight,HpHp_L0.1_mean,HpHp_L0.1_std,HpHp_L0.1_magnitude,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,label
0,146.055453,352.575175,58931.903411,232.135309,348.518825,59285.180744,650.351728,364.839501,57662.916172,6415.320351,381.100014,55515.473715,59471.354403,383.817678,55104.838944,146.055453,352.575175,58931.903411,232.135309,348.518825,59285.180744,650.351728,364.839501,57662.916172,6415.320351,381.100014,55515.473715,59471.354403,383.817678,55104.838944,86.502428,554.0,0.000000,554.000000,0.000000e+00,0.000000e+00,0.000000e+00,135.577740,554.0,0.000015,...,3.747075e+03,38985.832563,2.958823e+03,4.460872e+12,1.000000,554.0,0.000000,554.000000,0.000000e+00,0.0,0.0,1.000000,554.0,0.000000,554.000000,0.000000e+00,0.0,0.0,1.000000,554.0,0.000000,554.000000,0.000000e+00,0.0,0.0,1.000000,554.0,0.000000,554.000000,0.000000e+00,0.0,0.0,1.000000,554.0,0.000000,554.000000,0.000000e+00,0.0,0.0,1
1,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.0,0.000000,60.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,60.0,0.000000,...,0.000000e+00,1.000000,1.505914e+09,0.000000e+00,1.000000,60.0,0.000000,60.000000,0.000000e+00,0.0,0.0,1.000000,60.0,0.000000,60.000000,0.000000e+00,0.0,0.0,1.000000,60.0,0.000000,60.000000,0.000000e+00,0.0,0.0,1.000000,60.0,0.000000,60.000000,0.000000e+00,0.0,0.0,1.000000,60.0,0.000000,60.000000,0.000000e+00,0.0,0.0,2
2,175.348580,74.007747,0.185880,281.171798,74.016984,0.407329,809.265693,74.032642,0.786749,7333.704106,74.086616,20.064436,36414.069953,74.263424,158.190163,175.348580,74.007747,0.185880,281.171798,74.016984,0.407329,809.265693,74.032642,0.786749,7333.704106,74.086616,20.064436,36414.069953,74.263424,158.190163,175.291975,74.0,0.000002,95.268043,4.897779e-12,-1.078901e-28,-3.751296e-17,280.972821,74.0,0.000023,...,3.663085e+11,36307.262557,2.113774e+04,3.183117e+13,1.000000,74.0,0.000000,95.268043,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000,95.268043,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000,95.268043,0.000000e+00,0.0,0.0,1.020403,74.0,0.000000,95.268043,0.000000e+00,0.0,0.0,1.677593,74.0,0.000000,95.268043,0.000000e+00,0.0,0.0,2
3,33.587863,74.087364,2.093366,64.452935,74.119794,3.378410,159.182945,74.404193,58.408251,808.503099,75.184467,239.672595,2569.864737,75.088108,253.391223,33.587863,74.087364,2.093366,64.452935,74.119794,3.378410,159.182945,74.404193,58.408251,808.503099,75.184467,239.672595,2569.864737,75.088108,253.391223,1.000000,74.0,0.000000,74.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,74.0,0.000000,...,0.000000e+00,1.000000,1.505913e+09,0.000000e+00,1.000000,74.0,0.000000,74.000000,0.000000e+00,0.0,0.0,1.000000,74.

## Two Features


*   Two highest scoring features from the paper:
** Host_MAC&IP-10sec-Variance
** Host_IP-10sec-Variance

*   Two highest scoring features from this implementation:
** Host_MAC&IP-100ms-Packet Count
** Host_IP-100ms-Packet Count



### Decision Trees

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


X = s_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean']].copy()

#X = s_rand_combined_df.drop(['label'],axis=1).values

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9227392863885218

### K-Nearest Neighbour

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X = s_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean']].copy()

#X = s_rand_combined_df.drop(['label'],axis=1).values

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

nn = KNeighborsClassifier()

nn = nn.fit(X_train,Y_train)

Y_pred = nn.predict(X_test)

nn_acc = nn.score(X_test, Y_test)
nn_acc


0.9601189352463658

## Three Features


### Decision Tree

In [0]:


X = s_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean']].copy()

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc



0.9599773456673589

### K-Nearest Neighbour

In [0]:
X = s_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean']].copy()

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

nn = KNeighborsClassifier()

nn = nn.fit(X_train,Y_train)

Y_pred = nn.predict(X_test)

nn_acc = nn.score(X_test, Y_test)
nn_acc

0.98074381725505

## Ten Features

### Decision Tree

In [0]:
X = s_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean' ,'H_L3_mean', 'H_L5_variance', 
                        'MI_dir_L5_variance', 'MI_dir_L1_variance', 'H_L1_variance', 'H_L3_variance', 'MI_dir_L3_variance']].copy()

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier(random_state=42)

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9885784406267699

### K-Nearest Neighbour

In [0]:
X = s_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean' ,'H_L3_mean', 'H_L5_variance', 
                        'MI_dir_L5_variance', 'MI_dir_L1_variance', 'H_L1_variance', 'H_L3_variance', 'MI_dir_L3_variance']].copy()

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

nn = KNeighborsClassifier(n_neighbors=3)

nn = nn.fit(X_train,Y_train)

Y_pred = nn.predict(X_test)

nn_acc = nn.score(X_test, Y_test)
nn_acc


0.9815461582027563

# Feature Category Results

## Host-IP

In [0]:
X = s_rand_combined_df.iloc[:,16:30]

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc


0.999716820841986

## Host-MAC&IP

In [0]:

X = s_rand_combined_df.iloc[:,0:15]

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc


0.9996696243156503

## Channel

In [0]:


X = s_rand_combined_df.iloc[:,30:65]

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc


0.7754861242212573

## Network Jitter

In [0]:
X = s_rand_combined_df.iloc[:,65:80]

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9982065319992448

## Socket

In [0]:
X = s_rand_combined_df.iloc[:,80:115]

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.6983198036624504

# Time Interval Results

## 100 ms

In [0]:

x1 = s_rand_combined_df.iloc[:,12:15]
x2 = s_rand_combined_df.iloc[:,27:30]
x3 = s_rand_combined_df.iloc[:,58:65]
x4 = s_rand_combined_df.iloc[:,77:80]
x5 = s_rand_combined_df.iloc[:,108:115]

x_100_frames = [x1, x2, x3, x4, x5]

X = pd.concat(x_100_frames, axis=1)

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9996696243156503

## 500 ms

In [0]:
x1 = s_rand_combined_df.iloc[:,9:12]
x2 = s_rand_combined_df.iloc[:,24:27]
x3 = s_rand_combined_df.iloc[:,51:58]
x4 = s_rand_combined_df.iloc[:,74:77]
x5 = s_rand_combined_df.iloc[:,101:108]

x_500_frames = [x1, x2, x3, x4, x5]

X = pd.concat(x_100_frames, axis=1)

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9996224277893148

## 1.5 sec

In [0]:
x1 = s_rand_combined_df.iloc[:,6:9]
x2 = s_rand_combined_df.iloc[:,21:24]
x3 = s_rand_combined_df.iloc[:,44:51]
x4 = s_rand_combined_df.iloc[:,71:74]
x5 = s_rand_combined_df.iloc[:,94:101]

x_15_frames = [x1, x2, x3, x4, x5]

X = pd.concat(x_100_frames, axis=1)

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9996696243156503

## 10 sec

In [0]:
x1 = s_rand_combined_df.iloc[:,3:6]
x2 = s_rand_combined_df.iloc[:,18:21]
x3 = s_rand_combined_df.iloc[:,37:44]
x4 = s_rand_combined_df.iloc[:,68:71]
x5 = s_rand_combined_df.iloc[:,87:94]

x_10_frames = [x1, x2, x3, x4, x5]

X = pd.concat(x_100_frames, axis=1)

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9996224277893148

## 1 min

In [0]:
x1 = s_rand_combined_df.columns[0:3]
x2 = s_rand_combined_df.columns[15:18]
x3 = s_rand_combined_df.columns[30:37]
x4 = s_rand_combined_df.columns[65:68]
x5 = s_rand_combined_df.columns[80:87]

x_1_frames = [x1, x2, x3, x4, x5]

X = pd.concat(x_100_frames, axis=1)

Y = s_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9996224277893148

# Device-Based Modeling

# Balanced Vs Skewed

## Unbalanced: Skewed to Normal

In [0]:
#Numbers taken from the specificatioons in the paper

unb_benign_df = full_benign_df.sample(n=49683)
unb_mirai_df = full_mirai_df.sample(n=2870)
unb_bashlite_df = full_bashlite_df.sample(n=2798)

unb_benign_df = unb_benign_df.drop(columns=['Unnamed: 0'])
unb_mirai_df = unb_mirai_df.drop(columns=['Unnamed: 0'])
unb_bashlite_df = unb_bashlite_df.drop(columns=['Unnamed: 0'])

unb_benign_df['label'] = 0

unb_mirai_df['label'] = 1

unb_bashlite_df['label'] = 2

frames = [unb_benign_df, unb_mirai_df, unb_bashlite_df]
unb_combined_df = pd.concat(frames)

unb1_rand_combined_df = unb_combined_df.sample(frac=1).reset_index(drop=True)


In [0]:
X = unb1_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean']].copy()

Y = unb1_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.941948693243406

## Unbalanced: Skewed to Attacks

In [0]:
#Numbers taken from the specificatioons in the paper

unb_benign_df = full_benign_df.sample(n=5025)
unb_mirai_df = full_mirai_df.sample(n=29351)
unb_bashlite_df = full_bashlite_df.sample(n=28382)

unb_benign_df = unb_benign_df.drop(columns=['Unnamed: 0'])
unb_mirai_df = unb_mirai_df.drop(columns=['Unnamed: 0'])
unb_bashlite_df = unb_bashlite_df.drop(columns=['Unnamed: 0'])

unb_benign_df['label'] = 0

unb_mirai_df['label'] = 1

unb_bashlite_df['label'] = 2

frames = [unb_benign_df, unb_mirai_df, unb_bashlite_df]
unb_combined_df = pd.concat(frames)

unb2_rand_combined_df = unb_combined_df.sample(frac=1).reset_index(drop=True)

In [0]:
X = unb1_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean']].copy()

Y = unb1_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.942370227628568

## Balanced

In [0]:
#Numbers taken from the specificatioons in the paper

unb_benign_df = full_benign_df.sample(n=4000)
unb_mirai_df = full_mirai_df.sample(n=4000)
unb_bashlite_df = full_bashlite_df.sample(n=4000)

unb_benign_df = unb_benign_df.drop(columns=['Unnamed: 0'])
unb_mirai_df = unb_mirai_df.drop(columns=['Unnamed: 0'])
unb_bashlite_df = unb_bashlite_df.drop(columns=['Unnamed: 0'])

unb_benign_df['label'] = 0

unb_mirai_df['label'] = 1

unb_bashlite_df['label'] = 2

frames = [unb_benign_df, unb_mirai_df, unb_bashlite_df]
unb_combined_df = pd.concat(frames)

unb2_rand_combined_df = unb_combined_df.sample(frac=1).reset_index(drop=True)

In [0]:
X = unb1_rand_combined_df[['H_L5_mean', 'MI_dir_L5_mean', 'MI_dir_L3_mean']].copy()

Y = unb1_rand_combined_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

dt = DecisionTreeClassifier()

dt = dt.fit(X_train,Y_train)

Y_pred = dt.predict(X_test)

dt_acc = dt.score(X_test, Y_test)
dt_acc

0.9423100084306877